# VacationPy
----

#### Note
* This assignment was complete in collaboration with Ashley Parra.
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_df = pd.read_csv("../WeatherPy/City Weather.csv")
city_df.head()


,City,Lat,Lng,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,79.03,72,100,16.31,PF,1619229547
1,gizo,-8.1030,156.8419,84.33,71,79,7.65,SB,1619229614
2,punta arenas,-53.1500,-70.9167,42.80,93,40,3.44,CL,1619229607
3,bluff,-46.6000,168.3333,48.99,87,88,7.00,NZ,1619229496
4,kapaa,22.0752,-159.3190,77.00,73,75,17.27,US,1619229561


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [93]:
# Grabbing Locations
locations = [(city_df.iloc[i, 1], city_df.iloc[i, 2]) for i in city_df.index]
# Configure GMaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure(map_type='ROADMAP', zoom_level=2, center=(0,0))
heatmap_layer = gmaps.heatmap_layer(locations, weights=city_df["Humidity"])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [67]:
dropped_city_df = city_df
for i in city_df.index:
    if city_df.iloc[i, 3] < 70 or city_df.iloc[i, 3] > 80 or city_df.iloc[i, 5] != 0 or city_df.iloc[i, 6] > 10:
        dropped_city_df = dropped_city_df.drop([i])
dropped_city_df = dropped_city_df.reset_index()
dropped_city_df

,index,City,Lat,Lng,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date
0,24,maceio,-9.6658,-35.7353,73.4,100,0,3.44,BR,1619229449
1,64,geraldton,-28.7667,114.6000,75.2,43,0,6.91,AU,1619229643
2,178,ballina,-28.8667,153.5667,71.6,53,0,3.44,AU,1619229701
3,205,providencia,28.7167,-111.5833,72.0,51,0,5.99,MX,1619229720
4,257,changping,40.2186,116.2197,70.0,52,0,4.47,CN,1619229747
5,263,pitimbu,-7.4706,-34.8086,77.0,88,0,4.61,BR,1619229749
6,333,byron bay,-28.6500,153.6167,71.6,53,0,3.44,AU,1619229790
7,364,morondava,-20.2833,44.2833,74.3,79,0,6.17,MG,1619229800
8,402,quelimane,-17.8786,36.8883,73.4,83,0,2.30,MZ,1619229823
9,504,mehriz,31.5917,54.4318,75.2,22,0,4.61,IR,1619229875


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [97]:
# Setting up hotels dataframe
hotel_df = dropped_city_df
hotel_df = hotel_df.drop(columns="index")
hotel_df["Hotel Name"] = " "
# Setting precise city location
params = {"key": g_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    # Setting basic url
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    city = row['City']
    # Update address key value
    params['address'] = f"{city}"
    # Make request
    cities_lat_lng = requests.get(base_url, params=params)
    # Convert to json
    cities_lat_lng = cities_lat_lng.json()
    # Setting new locations
    hotel_df.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    hotel_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
# Setting url, query url, lat, and lng
lat = 0
lng = 0
url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=name&locationbias=circle:5000@"

for index, row in hotel_df.iterrows():
    # Grabbing lat and lng and query url
    lat = row["Lat"]
    lng = row["Lng"]
    query_url = f"{url}{lat},{lng}&key={g_key}"
    # Request search
    response = requests.get(query_url).json()
    # Setting hotel name
    hotel_df.loc[index, "Hotel Name"] = response["candidates"][0]["name"]
    
hotel_df


,City,Lat,Lng,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,maceio,-9.649849,-35.708949,73.4,100,0,3.44,BR,1619229449,Meridiano Hotel
1,geraldton,-28.775092,114.609858,75.2,43,0,6.91,AU,1619229643,Ocean Centre Hotel
2,ballina,54.114948,-9.155069,71.6,53,0,3.44,AU,1619229701,"Ocean Sands Hotel - Enniscrone, Co. Sligo, Ire..."
3,providencia,13.338761,-81.372937,72.0,51,0,5.99,MX,1619229720,Deep Blue
4,changping,40.220659,116.231203,70.0,52,0,4.47,CN,1619229747,Beijing Marriott Hotel Changping
5,pitimbu,-7.472545,-34.808110,77.0,88,0,4.61,BR,1619229749,HOTEL PONTA DE PEDRAS
6,byron bay,-28.647367,153.601977,71.6,53,0,3.44,AU,1619229790,Atlantic Byron Bay
7,morondava,-20.290419,44.299996,74.3,79,0,6.17,MG,1619229800,Palissandre Cote Ouest resort & SPA
8,quelimane,-17.850264,36.921858,73.4,83,0,2.30,MZ,1619229823,Pensao Bons Sinais
9,mehriz,31.577814,54.445158,75.2,22,0,4.61,IR,1619229875,Hotel baghe aho


In [98]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [100]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))